In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import util 
from utils import get_four_points
from skimage import morphology
from scipy.ndimage.morphology import binary_fill_holes
from skimage.measure import regionprops
from scipy.ndimage import label

In [2]:
# Read image

img = cv2.imread('graph.jpeg')
height, width = img.shape[:2]
print("width: ", width)
print("height: ", height)

img = cv2.resize(img, (720,800))
height, width = img.shape[:2]
print("width: ", width)
print("height: ", height)


width:  720
height:  1600
width:  720
height:  800


In [3]:
# Display Image

cv2.imshow('image',img)
cv2.waitKey(0)


-1

In [4]:
# Defining points
src_points = get_four_points(img)
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]]) 
print(dst_points)

[[  0.   0.]
 [720.   0.]
 [  0. 800.]
 [720. 800.]]


In [5]:
# Find homography using least square method (by default)
M_ls, _ = cv2.findHomography(src_points, dst_points)

dest_ls = cv2.warpPerspective(img, M_ls, (width, height))

In [6]:
# Drawing source points on original image

for i in range(0, 4):
    cv2.circle(img, (int(src_points[i][0]), int(src_points[i][1])), 10, (0, 255, 0), cv2.FILLED)

In [7]:
# Display Image

graph_img = dest_ls
Combined_img = cv2.hconcat([img, graph_img]) 
cv2.imwrite('Cropped_graph.png', graph_img)
cv2.imshow('Homography image',Combined_img)
cv2.waitKey(0)



-1

In [8]:

#convert into grayscale
gray_img = cv2.cvtColor(graph_img, cv2.COLOR_BGR2GRAY)

cv2.imshow('Grayscale Homography image',gray_img)
cv2.imwrite('Gray_Cropped_graph.png', gray_img)
cv2.waitKey(0)


thresh, binary_image = cv2.threshold(gray_img.astype(np.uint8), 0, 255,
                                         cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

cv2.imshow('Contour on Homography image',binary_image)
cv2.imwrite('Binary_Cropped_graph.png', binary_image)
cv2.waitKey(0)

-1

In [9]:
# Apply Guassian blur to smooth the image
img_smooth = cv2.GaussianBlur(binary_image, (9, 9), sigmaX = 0, sigmaY = 0) 
img_smooth = cv2.fastNlMeansDenoising(img_smooth)

In [10]:

find_con_gray = img_smooth

cnts = cv2.findContours(find_con_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

for c in cnts:
    cv2.drawContours(find_con_gray,[c], 0, (255,255,255), -1)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18,18))
opening = cv2.morphologyEx(find_con_gray, cv2.MORPH_OPEN, kernel, iterations=2)

cv2.imshow('find_con_gray', find_con_gray)
cv2.imwrite('Fill_shape_Binary_Cropped_graph.png', find_con_gray)
cv2.waitKey(0)
cv2.imshow('opening', opening)
cv2.imwrite('opening.png', opening)
cv2.waitKey(0)

kernel = np.ones((3,3), np.uint8)
Fill_shape_erosion = cv2.erode(find_con_gray, kernel, iterations=2)
cv2.imshow('erosion_Binary_Cropped_graph', Fill_shape_erosion)
cv2.imwrite('Fill_shape_erosion.png', Fill_shape_erosion)
cv2.waitKey(0)


kernel = np.ones((5,5), np.uint8)
Only_shape_dilate = cv2.dilate(opening, kernel, iterations=2)
cv2.imshow('dilation_Binary_Cropped_graph', Only_shape_dilate)
cv2.imwrite('Only_shape_dilate.png', Only_shape_dilate)
cv2.waitKey(0)


arrow =  Fill_shape_erosion - Only_shape_dilate 
cv2.imshow('Only_Arrow_Binary_Cropped_graph', arrow)
cv2.imwrite('arrow.png', arrow)
cv2.waitKey(0)

-1

In [11]:
blur_arrow = cv2.GaussianBlur(arrow,(11,11),0)
blur_arrow_result = cv2.threshold(blur_arrow, 100, 255, cv2.THRESH_BINARY)[1]
cv2.imshow("Result", blur_arrow_result)
cv2.waitKey(0)


-1

In [12]:
kernal = np.ones((3,3),np.uint8)
close_arrows = cv2.morphologyEx( blur_arrow_result,cv2.MORPH_OPEN,kernel)

cv2.imshow('gray', close_arrows)
cv2.waitKey(0)


-1

In [13]:
edges = cv2.Canny(image =Only_shape_dilate, threshold1 = 100, threshold2 = 100)
cv2.imshow('Edges of Dilation Inverted Binarize image',edges)
cv2.waitKey(0)

-1

In [14]:

# Apply Hough transform to greyscale image
Fill_shape_erosion_blur = cv2.medianBlur(Fill_shape_erosion,3)


circles = cv2.HoughCircles(Fill_shape_erosion_blur, cv2.HOUGH_GRADIENT,1,25,
                            param1=30,param2=30,minRadius=0,maxRadius=0)

In [15]:
circle_img = np.zeros((img.shape[:2]), dtype = 'uint8')

In [16]:
# Draw the circles
if circles is not None:
    circles= np.uint16(np.around(circles))
    for i in circles[0, :]:
    # draw the outer circle
    # cv2.circle(image, center_coordinates, radius, color, thickness)
        cv2.circle(circle_img, (i[0], i[1]), i[2], (255), 2)

In [17]:
cv2.imshow('Edges of Dilation Inverted Binarize image',circle_img)
cv2.waitKey(0)

-1

In [ ]:
# Canny Edge Detection
edges = cv2.Canny(image = coins_dilation, threshold1 = 100, threshold2 = 100)
cv2.imshow('Edges of Dilation Inverted Binarize image',edges)
cv2.waitKey(0)
cv2.imwrite("Edges of Dilation Inverted Binarize image.jpg", edges)